In [101]:
# Librerías
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from astroquery.mpc import MPC

# Cobs API

In [102]:
# Verificar la conexión a internet.
def verificar_conexion():
    try:
        requests.get("http://www.google.com", timeout=5)
        print('✅ Conectado a internet.')
        return True
    
    except requests.ConnectionError:
        print('🛑 Sin conexión a internet.')
        return False

In [103]:
# Conexión con la API de COBS
nombre_cometa =  '12P'
fecha_inicial = '1970-01-01 00:00'
Link_cops_API = f'https://cobs.si/api/obs_list.api?des={nombre_cometa}&format=json&from_date={fecha_inicial}&exclude_faint=False&exclude_not_accurate=True'

try: 
    if verificar_conexion():
        response = requests.get(Link_cops_API)

        if response.status_code == 200:
            content = response.json()
            print('✅ Base de datos actualizada.')
            
except requests.ConnectionError:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

✅ Conectado a internet.
✅ Base de datos actualizada.


In [104]:
# Creación del data frame Cometa
cometa_df = pd.DataFrame(content['objects'])
cometa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049 entries, 0 to 2048
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   type                       2049 non-null   object 
 1   obs_date                   2049 non-null   object 
 2   comet                      2049 non-null   object 
 3   observer                   2049 non-null   object 
 4   location                   938 non-null    object 
 5   extinction                 220 non-null    object 
 6   obs_method                 2049 non-null   object 
 7   comet_visibility           3 non-null      object 
 8   magnitude                  2049 non-null   object 
 9   conditions                 0 non-null      object 
 10  ref_catalog                2049 non-null   object 
 11  instrument_aperture        2049 non-null   object 
 12  instrument_type            2049 non-null   object 
 13  instrument_focal_ratio     1409 non-null   float

In [105]:
# Sin filtrar la información
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 2049
Variables: 47


In [106]:
# Base de datos arrojada por la API
cometa_df.sample(5)

,type,obs_date,comet,observer,location,extinction,obs_method,comet_visibility,magnitude,conditions,...,magnitude_error,comparison_star_magnitude,pixel_size_x,pixel_size_y,pixel_size_unit,obs_comment,obs_sky_quality,obs_sky_quality_method,reference_star_names,date_added
1826,C,2023-08-07 04:19:11,"{'type': 'P', 'name': '12P', 'fullname': '12P/...","{'first_name': 'Thomas', 'last_name': 'Lehmann...",Utah (remote),None,"{'key': 'Z', 'name': 'CCD Visual equivalent', ...",None,11.7,None,...,None,None,3.5,3.5,s,"CCD/G, mlim=18.5",None,NaN,,2023-08-25 13:22:23
474,V,2024-03-30 18:45:00,"{'type': 'P', 'name': '12P', 'fullname': '12P/...","{'first_name': 'Veikko', 'last_name': 'Mäkelä'...",Helsinki. Finland,None,"{'key': 'B', 'name': 'Simple Out-Out method', ...",None,4.9,None,...,None,None,None,None,None,twilight,4.0,1.0,"9 Ari, 14 Ari, 12 Ari",2024-03-30 19:44:06
422,C,2024-04-03 20:38:23,"{'type': 'P', 'name': '12P', 'fullname': '12P/...","{'first_name': 'Martin', 'last_name': 'Mašek',...",None,None,"{'key': 'D', 'name': 'Johnson/Bessel/Kron/Cous...",None,4.8,None,...,0.03,None,1.6,1.6,s,,None,NaN,,2024-04-04 13:13:18
1494,V,2023-10-22 18:28:47,"{'type': 'P', 'name': '12P', 'fullname': '12P/...","{'first_name': 'Gerhard', 'last_name': 'Scheer...",None,None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,12.4,None,...,None,None,None,None,None,,None,NaN,,2023-10-24 04:31:11
931,V,2024-02-12 19:12:00,"{'type': 'P', 'name': '12P', 'fullname': '12P/...","{'first_name': 'Edwin', 'last_name': 'van Dijk...",None,None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,7.5,None,...,None,None,None,None,None,,None,NaN,,2024-06-17 22:31:29


In [107]:
# Métodos de observación
cometa_df.obs_method.apply(
    lambda registro: f"{registro['key']}: {registro['name']}" if (registro is not None) and ('key' in registro) and ('name' in registro) else 'Datos faltantes'
).value_counts()

obs_method
S: In-Out method                             564
Z: CCD Visual equivalent                     430
M: Modified-Out method                       388
C: Unfiltered total CCD magnitude            353
V: Johnson/Bessel/Kron/Cousins V with CCD     91
B: Simple Out-Out method                      81
I: In-focus                                   33
H: Kron/Cousins I with CCD                    26
D: Johnson/Bessel/Kron/Cousins B with CCD     24
c: Unfiltered nuclear CCD magnitude           16
O: Out-of-focus (or extrafocal) method        12
E: Extrafocal-Extinction method               10
k: Kron/Cousins R with CCD                     8
-: Unknown                                     7
P: photographic                                3
A: Pogson                                      1
s: VSS method using image intensifier          1
Y: Wratten No. 15 with CCD                     1
Name: count, dtype: int64

In [108]:
# Tratamiento de los datos de interés
cometa_df['obs_method_key'] = cometa_df.obs_method.apply(lambda registro: registro['key'] if registro is not None and 'key' in registro else 'Dato faltante')
cometa_df['obs_date'] = pd.to_datetime(pd.to_datetime(cometa_df.obs_date).dt.date)
cometa_df['magnitude'] = pd.to_numeric(cometa_df.magnitude)

In [109]:
# Creación del data frame curva de luz cruda
curva_de_luz_cruda_df = cometa_df[['obs_method_key', 'obs_date', 'magnitude']].copy()
curva_de_luz_cruda_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049 entries, 0 to 2048
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  2049 non-null   object        
 1   obs_date        2049 non-null   datetime64[ns]
 2   magnitude       2049 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 48.2+ KB


In [110]:
# Con la información filtrada
filas,columnas = curva_de_luz_cruda_df.shape
print(f'Registros: {filas}\nVariables: {columnas}')

Registros: 2049
Variables: 3


In [111]:
# Data Frame de la curva de luz
curva_de_luz_cruda_df.sample(5)

,obs_method_key,obs_date,magnitude
1834,C,2023-08-04,14.1
1880,S,2023-07-26,12.1
1259,Z,2023-12-10,8.7
1630,S,2023-09-23,11.6
1151,C,2024-01-06,12.4


In [112]:
# Curva de luz cruda.
labels = {'obs_date':'Observation Date','magnitude':'Apparent total magnitude', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_cruda_df, x='obs_date', y='magnitude', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

# Perihelio Cobs API

In [113]:
# Conexión con la API de COBS
try: 
    Link_cops_API = f'https://cobs.si/api/comet.api?des={nombre_cometa}'

    if verificar_conexion():
        response = requests.get(Link_cops_API)

        if response.status_code == 200:
            perihelio = pd.to_datetime(response.json()['object']['perihelion_date'])
            print('✅ Perihelio del cometa obtenido.')
    
except requests.ConnectionError:
    print(f'🛑 Se presentó un error al cargar la base de datos.\nError: {response.status_code}\n{response.content}')

✅ Conectado a internet.
✅ Perihelio del cometa obtenido.


# MPC API usando astroquery.

In [114]:
# Creación de data frame Ephemeris (conexión con la API del MPC)
fecha_inicial = curva_de_luz_cruda_df.obs_date.min()
fecha_final = curva_de_luz_cruda_df.obs_date.max()
fechas = (fecha_final - fecha_inicial).days + 1 if (fecha_final - fecha_inicial).days <= 1441 else 1441

ephemeris = MPC.get_ephemeris(nombre_cometa, start = str(fecha_inicial), number = fechas)  # type: ignore

ephemeris_df = ephemeris.to_pandas()
ephemeris_df.columns = ephemeris_df.columns.str.lower().str.replace(' ', '_')
ephemeris_df

,date,ra,dec,delta,r,elongation,phase,v,proper_motion,direction
0,2022-06-06,277.227083,32.810000,6.834,7.397,120.2,6.8,22.0,20.59,300.2
1,2022-06-07,277.085000,32.878056,6.823,7.390,120.5,6.8,22.0,20.63,299.3
2,2022-06-08,276.941250,32.944444,6.812,7.383,120.7,6.8,22.0,20.66,298.4
3,2022-06-09,276.796250,33.008889,6.802,7.375,120.9,6.8,22.0,20.69,297.4
4,2022-06-10,276.649167,33.071389,6.791,7.368,121.2,6.8,22.0,20.72,296.5
...,...,...,...,...,...,...,...,...,...,...
843,2024-09-26,203.388750,-47.541944,3.114,2.581,49.6,17.2,13.6,65.44,89.9
844,2024-09-27,204.032083,-47.540833,3.134,2.593,49.2,17.0,13.7,64.87,89.8
845,2024-09-28,204.670000,-47.539444,3.154,2.605,48.7,16.8,13.7,64.32,89.8
846,2024-09-29,205.302500,-47.538056,3.174,2.617,48.3,16.6,13.8,63.77,89.8


In [115]:
# Info del data frame ephemeris
ephemeris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848 entries, 0 to 847
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           848 non-null    datetime64[ns]
 1   ra             848 non-null    float64       
 2   dec            848 non-null    float64       
 3   delta          848 non-null    float64       
 4   r              848 non-null    float64       
 5   elongation     848 non-null    float64       
 6   phase          848 non-null    float64       
 7   v              848 non-null    float64       
 8   proper_motion  848 non-null    float64       
 9   direction      848 non-null    float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 66.4 KB


In [116]:
# Dar a los datos el formato deseado
ephemeris_df.date = pd.to_datetime(ephemeris_df.date)
ephemeris_df.date = pd.to_datetime(ephemeris_df.date.dt.date)
ephemeris_df.dtypes

date             datetime64[ns]
ra                      float64
dec                     float64
delta                   float64
r                       float64
elongation              float64
phase                   float64
v                       float64
proper_motion           float64
direction               float64
dtype: object

In [117]:
# Creación del data frame ephemeris filtrada
ephemeris_filtrada_df = ephemeris_df[['date', 'delta','r', 'phase']].copy()
ephemeris_filtrada_df = ephemeris_filtrada_df.rename(columns = {'date':'obs_date'})
ephemeris_filtrada_df

,obs_date,delta,r,phase
0,2022-06-06,6.834,7.397,6.8
1,2022-06-07,6.823,7.390,6.8
2,2022-06-08,6.812,7.383,6.8
3,2022-06-09,6.802,7.375,6.8
4,2022-06-10,6.791,7.368,6.8
...,...,...,...,...
843,2024-09-26,3.114,2.581,17.2
844,2024-09-27,3.134,2.593,17.0
845,2024-09-28,3.154,2.605,16.8
846,2024-09-29,3.174,2.617,16.6


# Unión de las bases de datos.

In [118]:
# Unión de las bases de datos COBS y MPC
curva_de_luz_procesada_df = curva_de_luz_cruda_df.merge(ephemeris_filtrada_df, on='obs_date')
curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase
0,Z,2024-09-30,11.5,3.194,2.629,16.4
1,M,2024-09-23,11.3,3.054,2.545,17.8
2,C,2024-09-22,11.6,3.034,2.533,18.1
3,M,2024-09-17,11.0,2.932,2.472,19.1
4,C,2024-09-16,10.9,2.912,2.460,19.3
...,...,...,...,...,...,...
2044,C,2022-09-11,20.3,6.531,6.660,8.7
2045,C,2022-09-01,21.1,6.510,6.738,8.5
2046,C,2022-07-31,21.0,6.496,6.985,7.6
2047,C,2022-06-28,20.5,6.633,7.234,6.8


In [119]:
# Información del data frame curva de lus procesada
curva_de_luz_procesada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2049 entries, 0 to 2048
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  2049 non-null   object        
 1   obs_date        2049 non-null   datetime64[ns]
 2   magnitude       2049 non-null   float64       
 3   delta           2049 non-null   float64       
 4   r               2049 non-null   float64       
 5   phase           2049 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 96.2+ KB


In [120]:
# Reducción de la magnitud aparente y calculo del Delta t
beta = 0

curva_de_luz_procesada_df['magnitud_reducida'] = (
    curva_de_luz_cruda_df['magnitude'] 
    - 5 * np.log10(curva_de_luz_procesada_df['delta'] * curva_de_luz_procesada_df['r'])
    - (beta * curva_de_luz_procesada_df['phase'])
    )

curva_de_luz_procesada_df['delta_t'] = curva_de_luz_procesada_df.obs_date - perihelio  # type: ignore
curva_de_luz_procesada_df['delta_t'] = curva_de_luz_procesada_df.delta_t.apply(lambda delta_t: delta_t.days)

curva_de_luz_procesada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,Z,2024-09-30,11.5,3.194,2.629,16.4,6.879373,161
1,M,2024-09-23,11.3,3.054,2.545,17.8,6.847216,154
2,C,2024-09-22,11.6,3.034,2.533,18.1,7.171746,153
3,M,2024-09-17,11.0,2.932,2.472,19.1,6.698938,148
4,C,2024-09-16,10.9,2.912,2.460,19.3,6.624368,147
...,...,...,...,...,...,...,...,...
2044,C,2022-09-11,20.3,6.531,6.660,8.7,12.107730,-589
2045,C,2022-09-01,21.1,6.510,6.738,8.5,12.889440,-599
2046,C,2022-07-31,21.0,6.496,6.985,7.6,12.715938,-631
2047,C,2022-06-28,20.5,6.633,7.234,6.8,12.094557,-664


In [121]:
# Curva de luz reducida
labels = {'obs_date':'Observation Date','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='obs_date', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Reduced Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [122]:
# Curva de luz reducida
labels = {'delta_t':'t-Δt','magnitud_reducida':'Apparent total magnitude processed', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_procesada_df, x='delta_t', y='magnitud_reducida', color='obs_method_key', template= 'plotly_dark', labels= labels, title=f'Reduced Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [123]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_promediada_df = curva_de_luz_procesada_df.copy()
curva_de_luz_promediada_df['promedio_movil'] = curva_de_luz_promediada_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_promediada_df

,obs_method_key,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,Z,2024-09-30,11.5,3.194,2.629,16.4,6.879373,161,NaN
1,M,2024-09-23,11.3,3.054,2.545,17.8,6.847216,154,NaN
2,C,2024-09-22,11.6,3.034,2.533,18.1,7.171746,153,NaN
3,M,2024-09-17,11.0,2.932,2.472,19.1,6.698938,148,NaN
4,C,2024-09-16,10.9,2.912,2.460,19.3,6.624368,147,NaN
...,...,...,...,...,...,...,...,...,...
2044,C,2022-09-11,20.3,6.531,6.660,8.7,12.107730,-589,11.897710
2045,C,2022-09-01,21.1,6.510,6.738,8.5,12.889440,-599,12.104049
2046,C,2022-07-31,21.0,6.496,6.985,7.6,12.715938,-631,12.298138
2047,C,2022-06-28,20.5,6.633,7.234,6.8,12.094557,-664,12.299485


In [124]:
# Curva de luz Promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_promediada_df, x='obs_date', y='promedio_movil', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Average Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [125]:
# Curva de luz Promediada
labels = {'delta_t':'t - Δt','magnitud_reducida':'Max apparent total magnitude reduced', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_promediada_df, x='delta_t', y='promedio_movil', color='obs_method_key', template= 'plotly_dark', labels= labels, title= f'Average Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna (Envolvente inferior) v1 (Promedio corrido -> agrupación)

In [126]:
# Creación del data frame curva de luz agrupada
curva_de_luz_interna_v1_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').max()
curva_de_luz_interna_v1_df = curva_de_luz_interna_v1_df.reset_index()

curva_de_luz_interna_v1_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2022-06-06,C,21.5,6.834,7.397,6.8,12.981347,-686,12.490733
1,2022-06-28,C,20.5,6.633,7.234,6.8,12.094557,-664,12.299485
2,2022-07-31,C,21.0,6.496,6.985,7.6,12.715938,-631,12.298138
3,2022-09-01,C,21.1,6.510,6.738,8.5,12.889440,-599,12.104049
4,2022-09-11,C,20.3,6.531,6.660,8.7,12.107730,-589,11.897710
...,...,...,...,...,...,...,...,...,...
434,2024-09-16,C,10.9,2.912,2.460,19.3,6.624368,147,NaN
435,2024-09-17,M,11.0,2.932,2.472,19.1,6.698938,148,NaN
436,2024-09-22,C,11.6,3.034,2.533,18.1,7.171746,153,NaN
437,2024-09-23,M,11.3,3.054,2.545,17.8,6.847216,154,NaN


In [127]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v1_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='red', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [128]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v1_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='red', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente superior) v1 (Promedio corrido -> agrupación)

In [129]:
# Creación del data frame curva de luz agrupada
curva_de_luz_externa_v1_df = curva_de_luz_promediada_df.groupby(by = 'obs_date').min()
curva_de_luz_externa_v1_df = curva_de_luz_externa_v1_df.reset_index()
curva_de_luz_externa_v1_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2022-06-06,C,21.5,6.834,7.397,6.8,12.981347,-686,12.490733
1,2022-06-28,C,20.5,6.633,7.234,6.8,12.094557,-664,12.299485
2,2022-07-31,C,21.0,6.496,6.985,7.6,12.715938,-631,12.298138
3,2022-09-01,C,21.1,6.510,6.738,8.5,12.889440,-599,12.104049
4,2022-09-11,C,20.3,6.531,6.660,8.7,12.107730,-589,11.897710
...,...,...,...,...,...,...,...,...,...
434,2024-09-16,C,10.9,2.912,2.460,19.3,6.624368,147,NaN
435,2024-09-17,M,11.0,2.932,2.472,19.1,6.698938,148,NaN
436,2024-09-22,C,11.6,3.034,2.533,18.1,7.171746,153,NaN
437,2024-09-23,M,11.3,3.054,2.545,17.8,6.847216,154,NaN


In [130]:
# Gráfica de lus promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v1_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=6, line= dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [131]:
# Gráfica de lus promediada
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v1_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=6, line= dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz interna (Envolvente inferior) v2 (Agrupación  -> promedio corrido)

In [132]:
# Creación del data frame curva de luz agrupada
curva_de_luz_agrupada_max_v2_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').max()
curva_de_luz_agrupada_max_v2_df = curva_de_luz_agrupada_max_v2_df.reset_index()
curva_de_luz_agrupada_max_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,2022-06-06,C,21.5,6.834,7.397,6.8,12.981347,-686
1,2022-06-28,C,20.5,6.633,7.234,6.8,12.094557,-664
2,2022-07-31,C,21.0,6.496,6.985,7.6,12.715938,-631
3,2022-09-01,C,21.1,6.510,6.738,8.5,12.889440,-599
4,2022-09-11,C,20.3,6.531,6.660,8.7,12.107730,-589
...,...,...,...,...,...,...,...,...
434,2024-09-16,C,10.9,2.912,2.460,19.3,6.624368,147
435,2024-09-17,M,11.0,2.932,2.472,19.1,6.698938,148
436,2024-09-22,C,11.6,3.034,2.533,18.1,7.171746,153
437,2024-09-23,M,11.3,3.054,2.545,17.8,6.847216,154


In [133]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_agrupada_max_v2_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark',color='obs_method_key', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [134]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_interna_v2_df = curva_de_luz_agrupada_max_v2_df.copy()
curva_de_luz_interna_v2_df['promedio_movil'] = curva_de_luz_interna_v2_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_interna_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2022-06-06,C,21.5,6.834,7.397,6.8,12.981347,-686,NaN
1,2022-06-28,C,20.5,6.633,7.234,6.8,12.094557,-664,NaN
2,2022-07-31,C,21.0,6.496,6.985,7.6,12.715938,-631,NaN
3,2022-09-01,C,21.1,6.510,6.738,8.5,12.889440,-599,NaN
4,2022-09-11,C,20.3,6.531,6.660,8.7,12.107730,-589,NaN
...,...,...,...,...,...,...,...,...,...
434,2024-09-16,C,10.9,2.912,2.460,19.3,6.624368,147,6.773439
435,2024-09-17,M,11.0,2.932,2.472,19.1,6.698938,148,6.796469
436,2024-09-22,C,11.6,3.034,2.533,18.1,7.171746,153,6.862293
437,2024-09-23,M,11.3,3.054,2.545,17.8,6.847216,154,6.828520


In [135]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v2_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='red', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [136]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_interna_v2_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='red', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz Externa (Envolvente superior) v2 (Agrupación  -> promedio corrido)

In [137]:
# Creación del data frame curva de luz agrupada
curva_de_luz_agrupada_min_v2_df = curva_de_luz_procesada_df.groupby(by = 'obs_date').min()
curva_de_luz_agrupada_min_v2_df = curva_de_luz_agrupada_min_v2_df.reset_index()
curva_de_luz_agrupada_min_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,2022-06-06,C,21.5,6.834,7.397,6.8,12.981347,-686
1,2022-06-28,C,20.5,6.633,7.234,6.8,12.094557,-664
2,2022-07-31,C,21.0,6.496,6.985,7.6,12.715938,-631
3,2022-09-01,C,21.1,6.510,6.738,8.5,12.889440,-599
4,2022-09-11,C,20.3,6.531,6.660,8.7,12.107730,-589
...,...,...,...,...,...,...,...,...
434,2024-09-16,C,10.9,2.912,2.460,19.3,6.624368,147
435,2024-09-17,M,11.0,2.932,2.472,19.1,6.698938,148
436,2024-09-22,C,11.6,3.034,2.533,18.1,7.171746,153
437,2024-09-23,M,11.3,3.054,2.545,17.8,6.847216,154


In [138]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_agrupada_min_v2_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark',color='obs_method_key', labels= labels, title=f'Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [139]:
# Creación del data frame curva de luz promediada
numero_elementos_grupo = 7

curva_de_luz_externa_v2_df = curva_de_luz_agrupada_min_v2_df.copy()
curva_de_luz_externa_v2_df['promedio_movil'] = curva_de_luz_externa_v2_df.magnitud_reducida.rolling(window = numero_elementos_grupo).mean()
curva_de_luz_externa_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil
0,2022-06-06,C,21.5,6.834,7.397,6.8,12.981347,-686,NaN
1,2022-06-28,C,20.5,6.633,7.234,6.8,12.094557,-664,NaN
2,2022-07-31,C,21.0,6.496,6.985,7.6,12.715938,-631,NaN
3,2022-09-01,C,21.1,6.510,6.738,8.5,12.889440,-599,NaN
4,2022-09-11,C,20.3,6.531,6.660,8.7,12.107730,-589,NaN
...,...,...,...,...,...,...,...,...,...
434,2024-09-16,C,10.9,2.912,2.460,19.3,6.624368,147,6.773439
435,2024-09-17,M,11.0,2.932,2.472,19.1,6.698938,148,6.796469
436,2024-09-22,C,11.6,3.034,2.533,18.1,7.171746,153,6.862293
437,2024-09-23,M,11.3,3.054,2.545,17.8,6.847216,154,6.828520


In [140]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v2_df, x='obs_date', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [141]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_externa_v2_df, x='delta_t', y='promedio_movil', template= 'plotly_dark', labels= labels, title=f'Max Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='yellow', size=5, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz usando la mediada v1 (Mediana de cada día registrado).

In [142]:
# Creación del data frame curva de luz mediana v1
curva_de_luz_mediada_v1_df = curva_de_luz_procesada_df.groupby(by= 'obs_date').median(numeric_only= True)
curva_de_luz_mediada_v1_df = curva_de_luz_mediada_v1_df.reset_index()
curva_de_luz_mediada_v1_df

,obs_date,magnitude,delta,r,phase,magnitud_reducida,delta_t
0,2022-06-06,21.5,6.834,7.397,6.8,12.981347,-686.0
1,2022-06-28,20.5,6.633,7.234,6.8,12.094557,-664.0
2,2022-07-31,21.0,6.496,6.985,7.6,12.715938,-631.0
3,2022-09-01,21.1,6.510,6.738,8.5,12.889440,-599.0
4,2022-09-11,20.3,6.531,6.660,8.7,12.107730,-589.0
...,...,...,...,...,...,...,...
434,2024-09-16,10.9,2.912,2.460,19.3,6.624368,147.0
435,2024-09-17,11.0,2.932,2.472,19.1,6.698938,148.0
436,2024-09-22,11.6,3.034,2.533,18.1,7.171746,153.0
437,2024-09-23,11.3,3.054,2.545,17.8,6.847216,154.0


In [143]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v1_df, x='obs_date', y='magnitud_reducida', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [144]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v1_df, x='delta_t', y='magnitud_reducida', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Curva de luz usando la mediada v2 (Mediana de las dos curvas).

In [145]:
# Creación del data frame curva de luz mediana v2
curva_de_luz_mediada_v2_df = curva_de_luz_externa_v2_df.copy()
curva_de_luz_mediada_v2_df['mediana'] = (curva_de_luz_interna_v2_df['promedio_movil'] + curva_de_luz_externa_v2_df['promedio_movil'])/2
curva_de_luz_mediada_v2_df

,obs_date,obs_method_key,magnitude,delta,r,phase,magnitud_reducida,delta_t,promedio_movil,mediana
0,2022-06-06,C,21.5,6.834,7.397,6.8,12.981347,-686,NaN,NaN
1,2022-06-28,C,20.5,6.633,7.234,6.8,12.094557,-664,NaN,NaN
2,2022-07-31,C,21.0,6.496,6.985,7.6,12.715938,-631,NaN,NaN
3,2022-09-01,C,21.1,6.510,6.738,8.5,12.889440,-599,NaN,NaN
4,2022-09-11,C,20.3,6.531,6.660,8.7,12.107730,-589,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
434,2024-09-16,C,10.9,2.912,2.460,19.3,6.624368,147,6.773439,6.773439
435,2024-09-17,M,11.0,2.932,2.472,19.1,6.698938,148,6.796469,6.796469
436,2024-09-22,C,11.6,3.034,2.533,18.1,7.171746,153,6.862293,6.862293
437,2024-09-23,M,11.3,3.054,2.545,17.8,6.847216,154,6.828520,6.828520


In [146]:
# Gráfica de luz interna
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v2_df, x='obs_date', y='mediana', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

In [147]:
# Gráfica de luz interna
labels = {'delta_t':'t - Δt','magnitud_reducida':'Magnitude reduced'}
fig = px.scatter(curva_de_luz_mediada_v2_df, x='delta_t', y='mediana', template= 'plotly_dark', labels= labels, title=f'Mediated Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.update_traces(marker=dict(color='aquamarine', size=6, line=dict(width=1, color='DarkSlateGrey')))
fig.update_yaxes(autorange="reversed")
fig.show()

# Comparación de las curvas de luz v1 (Promedio corrido -> agrupación)

In [161]:
# Gráfica de luz promediada
labels = {'obs_date':'Observation Date','magnitud_reducida':'Magnitude reduced'}
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v1_df.obs_date, y=curva_de_luz_externa_v1_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.obs_date, y=curva_de_luz_interna_v1_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.obs_date, y=curva_de_luz_mediada_v1_df.magnitud_reducida, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.obs_date, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente_v2', marker=dict(color='green', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.obs_date, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo_v2', marker=dict(color='blue', line=dict(width=1, color='DarkSlateGrey'))))
# fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.obs_date, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana_v2', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.show()

In [ ]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v1_df.delta_t, y=curva_de_luz_externa_v1_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v1_df.delta_t, y=curva_de_luz_interna_v1_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_mediada_v1_df.delta_t, y=curva_de_luz_mediada_v1_df.magnitud_reducida, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='t - Δt', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.show()

# Comparación de las curvas de luz v2 (Agrupación -> promedio corrido)

In [150]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.obs_date, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.obs_date, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.obs_date, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='Observation Date', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.show()

In [151]:
# Gráfica de luz promediada
fig = go.Figure()
fig.add_trace(go.Scatter(x=curva_de_luz_externa_v2_df.delta_t, y=curva_de_luz_externa_v2_df.promedio_movil, mode='markers', name='Envolvente', marker=dict(color='yellow', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.delta_t, y=curva_de_luz_interna_v2_df.promedio_movil, mode='markers', name='Núcleo', marker=dict(color='red', line=dict(width=1, color='DarkSlateGrey'))))
fig.add_trace(go.Scatter(x=curva_de_luz_interna_v2_df.delta_t, y=curva_de_luz_mediada_v2_df.mediana, mode='markers', name='Mediana', marker=dict(color='aquamarine', line=dict(width=1, color='DarkSlateGrey'))))
fig.update_layout(template='plotly_dark')
fig.update_yaxes(autorange="reversed")
fig.update_layout(template='plotly_dark', xaxis_title='t - Δt', yaxis_title='Averaged Magnitude', title = f'Max/Min Averaged Lightcurve of comet {nombre_cometa} (Tsuchinshan-ATLAS)')
fig.show()